In [3]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile # für den Tor-Browser
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary # für den Tor-Browser
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
from stem import Signal # Um beim Tor-Surfer die IP/Idendity zu wechseln
from stem.control import Controller #Um beim Tor-Surfer die IP/Idendity zu wechseln
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

In [2]:
!pip install stem

  Created wheel for stem: filename=stem-1.8.0-cp37-none-any.whl size=436043 sha256=163804af94cb022131c95c8928b20d73d76130ea4c859e77dff0ff854714b918
  Stored in directory: C:\Users\Zoe\AppData\Local\pip\Cache\wheels\02\3a\ee\1094b166e029353f892c0b121aa02f48aff5e658396924bc2a
Successfully built stem


In [8]:
dff_g=pd.read_csv(folderpath+"/BRFrames/alle_Adressen_ohne_Stadt")

In [9]:
# Nun das Gemeinde-Dataframe miot 16740 Rows in 1000-er-Stücke teilen. (würde sicher auch automatisiert gehen...)
dff1=dff_g.iloc[:1000,:]
dff2=dff_g.iloc[1000:2000,:]
dff3=dff_g.iloc[2000:3000,:]
dff33=dff_g.iloc[2500:3000,:]
dff34=dff_g.iloc[2530:3000,:]
dff4=dff_g.iloc[3080:3200,:]
dff5=dff_g.iloc[3201:3400,:]
dff6=dff_g.iloc[3400:3800,:]
dff7=dff_g.iloc[3960:4200,:]
dff8=dff_g.iloc[4200:4600,:]
dff9=dff_g.iloc[4600:5000,:]
dff10=dff_g.iloc[5000:5400,:]
dff11=dff_g.iloc[5700:6200,:]
dff12=dff_g.iloc[6200:6600,:]
dff13=dff_g.iloc[6600:7000,:]
dff14=dff_g.iloc[7000:14000,:]
dff15=dff_g.iloc[14000:15000,:]
dff16=dff_g.iloc[15000:16000,:]
dff17=dff_g.iloc[16000:,:]
#[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17]

In [4]:

listeadress=[]
counter=600
nummer=6

df_list=[dff7,dff8,dff9,dff10,dff11,dff12,dff13,dff14,dff15,dff16,dff17]

for df in tqdm(df_list):
    #Wir starten den Browser auf
    driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # jetzt via Tor mit unterschiedlichen IP's
    binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
    #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
    driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
    time.sleep(5)
    #Hier drücke ich den Cookie-Banner weg
    cookie=driver.find_element_by_id("j_idt198")
    time.sleep(4)            
    cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
    time.sleep(2)
    #Ich öffne das zusätzliche Panel, ohne dass man dies auf der Seite sieht
    #su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
    #time.sleep(2)
    #driver.execute_script('arguments[0].click();', su_fel_ad)
    #time.sleep(2)
    # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
    #filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
    #filt.send_keys('Domiziladresse')
    #filt.send_keys(Keys.ENTER)
    #time.sleep(2)
    # Hier klappe ich die Zusätzlichen Suchfelder auf
    driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
    time.sleep(3)
    for adress in tqdm(df["Adresse"]): #die Adressen durchgehen
        # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
        su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
        time.sleep(1)
        driver.execute_script('arguments[0].click();', su_fel_ad)
        time.sleep(1)
        filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
        time.sleep(1)
        filt.send_keys('Domiziladresse')
        filt.send_keys(Keys.ENTER)
        time.sleep(1)
    
        search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
        search_adress.send_keys(adress) #Adresse Eingeben
        search_adress.send_keys(Keys.ENTER) #Enter drücken
        time.sleep(5)
        page = driver.page_source.encode("utf-8") #Page in Variable speichern

        with open (folderpath+"/HRSeiten_Gemeinden/frontpage"+str(nummer)+"_"+str(counter)+".htm", "wb+") as file:
            file.write(page)
            file.close()
            
        pagename = "frontpage"+str(nummer)+"_"+str(counter)+".htm"        
        counter=counter+1 

        clear_all=driver.find_element_by_id("idSucheForm:j_idt167").click()
        time.sleep(3)
               

        
        minidictadress={"Adresse":adress, "Dokument":pagename}
        listeadress.append(minidictadress)
    nummer=nummer+1
    driver.close() # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
    time.sleep(300) # Nun Pause.

        

  0%|          | 0/11 [00:00<?, ?it/s]

NoSuchElementException: Message: Unable to locate element: [id="j_idt198"]


### Versuch: IP-Wechsel von Tor nach jedem Zugriff
Trotz dem Tor-Browser und einem immer wieder schliessenden und wieder öffnenden Browser kann ich nicht nur noch Päckchen an höchstens 4000 Adressen runterladen. Definitiv zu wenig wenn ich noch total ca 10000 abfragen will. Nun noch einen Versuch mit wechselnder IP beim Tor-Browser. 
https://stem.torproject.org/faq.html

In [14]:
# Hiermit sollte die IP des Tor-Browsers gewechselt werden. Geht aber nicht
with Controller.from_port(port = 9051) as controller:
            controller.authenticate()
            controller.signal(Signal.NEWNYM)       


IncorrectSocketType: unable to use the control socket

In [11]:
torexe = os.popen(r'C:\Users\Zoe\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')
profile = FirefoxProfile(r'C:\Users\Zoe\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9050)
profile.set_preference("network.proxy.socks_remote_dns", False)
profile.update_preferences()
driver = webdriver.Firefox(firefox_profile= profile, executable_path=r"C:\geckodriver.exe")
driver.get("http://check.torproject.org")

In [12]:
#offenbar hatte ich gar keinen Zugriff bisher auf den Tor. Nun gehts. Mal schauen obs jetzt geht.
listeadress=[]
counter=600
nummer=6

df_list=[dff7,dff8,dff9,dff10,dff11,dff12,dff13,dff14,dff15,dff16,dff17]

for df in tqdm(df_list):
    #Wir starten den Tor-Browser angehen auf
    torexe = os.popen(r'C:\Users\Zoe\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')
    profile = FirefoxProfile(r'C:\Users\Zoe\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
    profile.set_preference('network.proxy.type', 1)
    profile.set_preference('network.proxy.socks', '127.0.0.1')
    profile.set_preference('network.proxy.socks_port', 9050)
    profile.set_preference("network.proxy.socks_remote_dns", False)
    profile.update_preferences()
    driver = webdriver.Firefox(firefox_profile= profile, executable_path=r"C:\geckodriver.exe")
    
    #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
    driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
    time.sleep(5)
    #Hier drücke ich den Cookie-Banner weg
    cookie=driver.find_element_by_id("j_idt198")
    time.sleep(4)            
    cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
    time.sleep(2)
    driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
    time.sleep(3)
    for adress in tqdm(df["Adresse"]): #die Adressen durchgehen
        # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
        su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
        time.sleep(1)
        driver.execute_script('arguments[0].click();', su_fel_ad)
        time.sleep(1)
        filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
        time.sleep(1)
        filt.send_keys('Domiziladresse')
        filt.send_keys(Keys.ENTER)
        time.sleep(1)
    
        search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
        search_adress.send_keys(adress) #Adresse Eingeben
        search_adress.send_keys(Keys.ENTER) #Enter drücken
        time.sleep(5)
        page = driver.page_source.encode("utf-8") #Page in Variable speichern

        with open (folderpath+"/HRSeiten_Gemeinden/frontpage"+str(nummer)+"_"+str(counter)+".htm", "wb+") as file:
            file.write(page)
            file.close()
            
        pagename = "frontpage"+str(nummer)+"_"+str(counter)+".htm"        
        counter=counter+1 

        clear_all=driver.find_element_by_id("idSucheForm:j_idt167").click()
        time.sleep(3)
        
        
        
        minidictadress={"Adresse":adress, "Dokument":pagename}
        listeadress.append(minidictadress)
    nummer=nummer+1
    driver.close() # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
    time.sleep(300) # Nun Pause.




  0%|          | 0/11 [00:00<?, ?it/s]


  0%|          | 0/240 [00:00<?, ?it/s]

SocketError: [WinError 10061] No connection could be made because the target machine actively refused it